In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [11]:
# Procuring Stock Data
tickers=['AAPL']
aapl=yf.download(tickers,start='2010-01-01',end='2023-01-01')

[*********************100%%**********************]  1 of 1 completed


In [12]:
aapl.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.478998,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.490199,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.386964,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.375155,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.417541,447610800


In [14]:
# WILLIAMS %R CALCULATION

def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

aapl['wr_14'] = get_wr(aapl['High'], aapl['Low'], aapl['Close'], 14)
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,wr_14
Date,,,,,,,
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,-89.080172
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,-93.835306
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339409,85438400,-99.294613
2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,-84.481336
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.207779,77034200,-83.153568


In [15]:
# MACD CALCULATION

def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist

aapl['macd'] = get_macd(aapl['Close'], 26, 12, 9)[0]
aapl['macd_signal'] = get_macd(aapl['Close'], 26, 12, 9)[1]
aapl['macd_hist'] = get_macd(aapl['Close'], 26, 12, 9)[2]
aapl = aapl.dropna()
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,wr_14,macd,macd_signal,macd_hist
Date,,,,,,,,,,
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,-89.080172,-3.670694,-2.570162,-1.100532
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.307236,69007800,-93.835306,-3.936230,-2.843375,-1.092855
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.339409,85438400,-99.294613,-4.417705,-3.158241,-1.259464
2022-12-29,127.989998,130.479996,127.730003,129.610001,128.889572,75703700,-84.481336,-4.459798,-3.418553,-1.041245
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.207779,77034200,-83.153568,-4.416427,-3.618128,-0.798299


In [17]:
# TRADING STRATEGY

def implement_wr_macd_strategy(prices, wr, macd, macd_signal):
    buy_price = []
    sell_price = []
    wr_macd_signal = []
    signal = 0

    for i in range(len(wr)):
        if wr[i-1] > -50 and wr[i] < -50 and macd[i] > macd_signal[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)

        elif wr[i-1] < -50 and wr[i] > -50 and macd[i] < macd_signal[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)

        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            wr_macd_signal.append(0)

    return buy_price, sell_price, wr_macd_signal

buy_price, sell_price, wr_macd_signal = implement_wr_macd_strategy(aapl['Close'], aapl['wr_14'], aapl['macd'], aapl['macd_signal'])

In [19]:
# POSITION

position = []
for i in range(len(wr_macd_signal)):
    if wr_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(aapl['Close'])):
    if wr_macd_signal[i] == 1:
        position[i] = 1
    elif wr_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]

close_price = aapl['Close']
wr = aapl['wr_14']
macd_line = aapl['macd']
signal_line = aapl['macd_signal']
wr_macd_signal = pd.DataFrame(wr_macd_signal).rename(columns = {0:'wr_macd_signal'}).set_index(aapl.index)
position = pd.DataFrame(position).rename(columns = {0:'wr_macd_position'}).set_index(aapl.index)

frames = [close_price, wr, macd_line, signal_line, wr_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy.head()

,Close,wr_14,macd,macd_signal,wr_macd_signal,wr_macd_position
Date,,,,,,
2010-01-22,7.062500,-96.798768,-0.072355,-0.040464,0,1
2010-01-25,7.252500,-67.932772,-0.083188,-0.049009,0,1
2010-01-26,7.355000,-52.256679,-0.082552,-0.055717,0,1
2010-01-27,7.424286,-41.707433,-0.075585,-0.059691,-1,0
2010-01-28,7.117500,-88.417704,-0.093738,-0.066500,0,0


In [24]:
# BACKTESTING

aapl_ret = pd.DataFrame(np.diff(aapl['Close'])).rename(columns = {0:'returns'})
wr_macd_strategy_ret = []

for i in range(len(aapl_ret)):
    try:
        returns = aapl_ret['returns'][i] * strategy['wr_macd_position'][i]
        wr_macd_strategy_ret.append(returns)
    except:
        pass

wr_macd_strategy_ret_df = pd.DataFrame(wr_macd_strategy_ret).rename(columns = {0:'wr_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / aapl['Close'][0])
wr_macd_investment_ret = []

for i in range(len(wr_macd_strategy_ret_df['wr_macd_returns'])):
    returns = number_of_stocks * wr_macd_strategy_ret_df['wr_macd_returns'][i]
    wr_macd_investment_ret.append(returns)

wr_macd_investment_ret_df = pd.DataFrame(wr_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(wr_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the Williams%R & MACD strategy by investing $100k in AAPL from 2010-01-01 to 2023-01-01 : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the W%R MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the Williams%R & MACD strategy by investing $100k in AAPL from 2010-01-01 to 2023-01-01 : 1699342.57
Profit percentage of the W%R MACD strategy : 1699%
